# Data Science Academy / Week 3 Homework
# Titus Teodorescu

## Part 1: Parse the JSON file to a pandas dataframe

In [4]:
# load packages
import numpy as np
import pandas as pd
import json
from datetime import datetime
# load json file
with open('../data_for_week3/homework/week3_homework.json') as f:
    txt = json.load(f)
# create df with eventSequence data
df = pd.DataFrame(txt['gameLog'][0]['eventSequence'])
# add a column for sessionID 
df['sessionID']=[int(txt['gameLog'][0]['sessionID'])] * len(txt['gameLog'][0]['eventSequence'])
# add a column for teamID 
df['teamID']=[txt['gameLog'][0]['teamID']] * len(txt['gameLog'][0]['eventSequence'])
# set the type of selected columns to str
df[['eventName', 'eventBy', 'eventTo', 'eventResult', 'eventLocation', 'teamID']] = df[['eventName', 'eventBy', 'eventTo', 'eventResult', 'eventLocation', 'teamID']].astype("string")
# set the type of selected columns to datetime
df[['eventStartTime', 'eventEndTime']] = df[['eventStartTime', 'eventEndTime']].apply(pd.to_datetime)

peak memory: 103.40 MiB, increment: 0.02 MiB


In [2]:
# show types of columns
df.dtypes

eventName                      string
eventStartTime    datetime64[ns, UTC]
eventEndTime      datetime64[ns, UTC]
eventBy                        string
eventTo                        string
eventResult                    string
eventLocation                  string
eventExtData                   object
sessionID                       int64
teamID                         string
dtype: object

In [3]:
# display the first 5 rows
df[:5]

,eventName,eventStartTime,eventEndTime,eventBy,eventTo,eventResult,eventLocation,eventExtData,sessionID,teamID
0,chat,2019-11-06 14:18:31+00:00,2019-11-06 14:18:31+00:00,jiangang,others,hi,slide1-step0,{},7369,hao_jiangang
1,question,2019-11-06 14:18:42+00:00,2019-11-06 14:18:42+00:00,jiangang,cbal-1-0,1,slide2-step0,{},7369,hao_jiangang
2,question,2019-11-06 14:18:42+00:00,2019-11-06 14:18:42+00:00,jiangang,cbal-1-1,dfsa,slide2-step0,{},7369,hao_jiangang
3,chat,2019-11-06 14:18:44+00:00,2019-11-06 14:18:44+00:00,jiangang,others,ok,slide2-step0,{},7369,hao_jiangang
4,chat,2019-11-06 14:18:46+00:00,2019-11-06 14:18:46+00:00,jiangang,others,how are you,slide2-step0,{},7369,hao_jiangang


## Part 2: Convert the XML file to a pandas dataframe and save it as a csv file.

### Approach #1: using pandas read_xml call

In [66]:
# you need pandas 1.3.0 or higher in order to use read_xml
# I was not able to install pandas 1.3.0 in anaconda, but I installed it outside anaconda

with open('../data_for_week3/homework/week3_homework.xml') as f:
    txt = f.read()
# create df with eventSequence data
# you need pandas 1.3.0 to call read_xml; The line below will fail in anaconda if you don't have pandas 1.3.0 or higher installed
df = pd.read_xml(txt, xpath="//event")
df_2 = pd.read_xml(txt, xpath="/gameLog/session")

# add a column for sessionID
# df_2['sessionID'][0] gives the value of the sessionID
# df.shape[0] gives the number of rows in df
df['sessionID']=[int(df_2['sessionID'][0])] * df.shape[0]
# add a column for teamID
df['teamID']=[str(df_2['teamID'][0])] * df.shape[0]
# set the type of selected columns to str
df[['eventName', 'eventBy', 'eventTo', 'eventResult', 'eventLocation', 'teamID']] = df[['eventName', 'eventBy', 'eventTo', 'eventResult', 'eventLocation', 'teamID']].astype("string")
# set the type of selected columns to datetime
df[['eventStartTime', 'eventEndTime']] = df[['eventStartTime', 'eventEndTime']].apply(pd.to_datetime)
# show types of columns
print(df.dtypes)
# display the first 5 rows
print(df[:5])
# save df to a csv file
with open('approach_01.csv', mode="w") as f:
    f.write(df.to_csv(index=False))

### Approach #2: using the XML package

In [4]:
# load package
import xml.etree.ElementTree as et
# parse xml file
xml_file_name = '../data_for_week3/homework/week3_homework.xml'
tree = et.parse(xml_file_name)
root = tree.getroot()
# set up the columns
eventName = list(map(lambda x: x.text, root.findall("./session/eventSequence/event/eventName")))
eventStartTime = list(map(lambda x: x.text, root.findall("./session/eventSequence/event/eventStartTime")))
eventEndTime = list(map(lambda x: x.text, root.findall("./session/eventSequence/event/eventEndTime")))
eventBy = list(map(lambda x: x.text, root.findall("./session/eventSequence/event/eventBy")))
eventTo = list(map(lambda x: x.text, root.findall("./session/eventSequence/event/eventTo")))
eventResult = list(map(lambda x: x.text, root.findall("./session/eventSequence/event/eventResult")))
eventLocation = list(map(lambda x: x.text, root.findall("./session/eventSequence/event/eventLocation")))
sessionID = list(map(lambda x: x.text, root.findall("./session/sessionID"))) * len(eventName)
teamID = list(map(lambda x: x.text, root.findall("./session/teamID"))) * len(eventName)
# initialize data of lists.
data = {'eventName':eventName,
        'eventStartTime':eventStartTime,
        'eventEndTime':eventEndTime,
        'eventBy':eventBy,
        'eventTo':eventTo,
        'eventResult':eventResult,
        'eventLocation':eventLocation,
        'sessionID':sessionID,
        'teamID':teamID}
# ceate DataFrame
df = pd.DataFrame(data)
# set the type of selected columns to str
df[['eventName', 'eventBy', 'eventTo', 'eventResult', 'eventLocation', 'teamID']] = df[['eventName', 'eventBy', 'eventTo', 'eventResult', 'eventLocation', 'teamID']].astype("string")
# set the type of selected columns to datetime
df[['eventStartTime', 'eventEndTime']] = df[['eventStartTime', 'eventEndTime']].apply(pd.to_datetime)
# show types of columns
# print(df.dtypes)
# display the first 5 rows
# print(df[:5])
# save df to a csv file
with open('approach_02.csv', mode="w") as f:
    f.write(df.to_csv(index=False))

### Approach #3: using xslt in lxml

In [5]:
# create the xslt transformation
XSL= '''
<xsl:stylesheet version="1.0"
                xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
	<xsl:output method="text"
	            omit-xml-declaration="yes"/>
	<xsl:variable name="sessionID"
	              select="/gameLog/session/sessionID"/>
	<xsl:variable name="teamID"
	              select="/gameLog/session/teamID"/>
	<xsl:template match="/">
		<xsl:text>eventName,eventStartTime,eventEndTime,eventBy,eventTo,eventResult,eventLocation,sessionID,teamID&#xa;</xsl:text>
		<xsl:for-each select="/gameLog/session/eventSequence/event">
			<xsl:value-of select="eventName"/>
			<xsl:text>,</xsl:text>
			<xsl:value-of select="eventStartTime"/>
			<xsl:text>,</xsl:text>
			<xsl:value-of select="eventEndTime"/>
			<xsl:text>,</xsl:text>
			<xsl:value-of select="eventBy"/>
			<xsl:text>,</xsl:text>
			<xsl:value-of select="eventTo"/>
			<xsl:text>,</xsl:text>
			<xsl:value-of select="eventResult"/>
			<xsl:text>,</xsl:text>
			<xsl:value-of select="eventLocation"/>
			<xsl:text>,</xsl:text>
			<xsl:value-of select="$sessionID"/>
			<xsl:text>,</xsl:text>
			<xsl:value-of select="$teamID"/>
			<xsl:text>&#xa;</xsl:text>
		</xsl:for-each>
	</xsl:template>
</xsl:stylesheet>
'''

In [6]:
from lxml import etree
# load xml file
dom = etree.parse('../data_for_week3/homework/week3_homework.xml')
# load xslt file
transform = etree.XSLT(etree.fromstring(XSL))
# apply xslt transformation to the xml file and save it to a csv file
# s = str(transform(dom))
# line = s.replace(‘\n’,’’)
# line.splitlines()
# create the csv file using the xslt transformation
with open('approach_03.csv', mode="w") as f:
    f.write(str(transform(dom)))

In [7]:
# create the data frame from the csv file
df = pd.read_csv("approach_03.csv")
# set the type of selected columns to str
df[['eventName', 'eventBy', 'eventTo', 'eventResult', 'eventLocation', 'teamID']] = df[['eventName', 'eventBy', 'eventTo', 'eventResult', 'eventLocation', 'teamID']].astype("string")
# set the type of selected columns to datetime
df[['eventStartTime', 'eventEndTime']] = df[['eventStartTime', 'eventEndTime']].apply(pd.to_datetime)
# show types of columns
print(df.dtypes)
# display the first 5 rows
print(df[:5])

eventName                      string
eventStartTime    datetime64[ns, UTC]
eventEndTime      datetime64[ns, UTC]
eventBy                        string
eventTo                        string
eventResult                    string
eventLocation                  string
sessionID                       int64
teamID                         string
dtype: object
  eventName            eventStartTime              eventEndTime   eventBy  \
0      chat 2019-11-06 14:18:31+00:00 2019-11-06 14:18:31+00:00  jiangang   
1  question 2019-11-06 14:18:42+00:00 2019-11-06 14:18:42+00:00  jiangang   
2  question 2019-11-06 14:18:42+00:00 2019-11-06 14:18:42+00:00  jiangang   
3      chat 2019-11-06 14:18:44+00:00 2019-11-06 14:18:44+00:00  jiangang   
4      chat 2019-11-06 14:18:46+00:00 2019-11-06 14:18:46+00:00  jiangang   

    eventTo  eventResult eventLocation  sessionID        teamID  
0    others           hi  slide1-step0       7369  hao_jiangang  
1  cbal-1-0            1  slide2-step0       7369